In [3]:
# Auto-reload internal modules
%load_ext autoreload
%autoreload 2

# External modules
import numpy as np

# Internal modules
from cosmotd.domain_walls import plot_domain_wall_simulation
from cosmotd.cosmic_strings import plot_cosmic_string_simulation
from cosmotd.charged_domain_walls import plot_charged_domain_wall_simulation
from cosmotd.pentavac import plot_pentavac_simulation
from cosmotd.single_axion import plot_single_axion_simulation
from cosmotd.companion_axion import plot_companion_axion_simulation
from cosmotd.plot import *
from cosmotd.parameters import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Size of simulation grid
N = 50
# Model parameters
beta = 1
eta_phi = 1
eta_sigma = np.sqrt(3) / 2
lam_phi = 2
lam_sigma = 2
charge_density = 0.1
# RNG seed
seed = 486761876
# Run time
run_time = None
# Plotter
plotter = MplNotebookPlotter

In [ ]:
%%time
# Domain walls simulation
# file_name = "domain_wall_M200_N200_np23213241.ctdd"
file_name = None
plot_domain_wall_simulation(200, 200, DX, DT, ALPHA_2D, eta_phi, RADIATION_ERA, 5, MplNotebookPlotter, run_time, file_name, seed)

In [ ]:
%%time
# Cosmic strings simulation
file_name = None
plot_cosmic_string_simulation(200, 200, DX, DT, ALPHA_2D, eta_phi, RADIATION_ERA, 5, MplNotebookPlotter, run_time, file_name, seed)

In [ ]:
%%time
# Charged domain walls simulation
# file_name = "charged_domain_walls_rho0.1_M200_N200_np23213241.ctdd"
file_name = None
plot_charged_domain_wall_simulation(
    200,
    200,
    DX,
    DT,
    ALPHA_2D,
    beta,
    eta_phi,
    eta_sigma,
    lam_phi,
    lam_sigma,
    charge_density,
    RADIATION_ERA,
    MplMultiPlotter,
    run_time,
    file_name,
    seed
)

In [ ]:
%%time
# Pentavac model simulation
file_name = None
epsilon = 0.1
plot_pentavac_simulation(200, 200, DX, DT, ALPHA_2D, epsilon, RADIATION_ERA, MplMultiPlotter, run_time, file_name, seed)

In [7]:
%%time
file_name = None
# Single axion
lam = 5
color_anomaly = 5
K = 0.001 * lam
t0 = 750 * DT
growth = 2
plot_single_axion_simulation(300, 300, DX, DT, ALPHA_2D, eta_phi, RADIATION_ERA, lam, color_anomaly, K, t0, growth, MplMultiPlotter, 6000, file_name, seed)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6001/6001 [09:11<00:00, 10.88it/s]

CPU times: total: 8min 38s
Wall time: 9min 11s


In [16]:
%%time
file_name = None
# file_name = "companion_axion_M200_N200_np23213241.ctdd"
# Companion axion
lam = 5
# n = 3
# n_prime = 1 / 2
# m = 13 / 2
# m_prime = 3 / 2
n = 3
n_prime = 1
m = 1
m_prime = 1
kappa = 0.04
K = 0.001 * lam
t0 = 750 * DT
n_growth = 2
m_growth = 2
s0 = t0
plot_companion_axion_simulation(300, 300, DX, DT, ALPHA_2D, eta_phi, RADIATION_ERA, lam, n, n_prime, m, m_prime, K, kappa, t0, s0, n_growth, m_growth, MplMultiPlotter, 6000, file_name, seed)
# for _ in range(5):
#     plot_companion_axion_simulation(300, 300, DX, DT, ALPHA_2D, eta_phi, RADIATION_ERA, lam, n, n_prime, m, m_prime, K, kappa, t0, s0, n_growth, m_growth, Plotter, 6000, file_name, None)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6001/6001 [15:37<00:00,  6.40it/s]

CPU times: total: 12min 42s
Wall time: 15min 38s
